In [1]:
# Importing dependencies
import  pandas as pd
import gmaps
import  requests

#import API key
#from config import g_key

In [16]:
# Storing the cities.csv file
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Wrzesnia,PL,2021-12-04 01:17:11,52.3251,17.5652,33.87,68,100,13.24
1,1,Jamestown,US,2021-12-04 01:17:12,42.0970,-79.2353,35.10,84,90,5.01
2,2,Dikson,RU,2021-12-04 01:17:12,73.5069,80.5464,-22.04,95,74,12.55
3,3,Gat,IL,2021-12-04 01:17:13,31.6100,34.7642,60.40,58,100,3.24
4,4,Zhigansk,RU,2021-12-04 01:17:14,66.7697,123.3711,-25.28,90,100,19.64


In [17]:
# Checking data types for DataFrame
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [18]:
# Configuring gmaps to use api key
gmaps.configure(api_key=g_key)

In [35]:
# Heatmap of temperature
# Getting the lat and long
locations = city_data_df[['Lat', 'Lng']]
# Getting the maximum temperature
max_temp = city_data_df['Max Temp']

# Assigning figure variable
# Adding zoom level and center
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assigning the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=[max(temp,0) for temp in max_temp],
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
# Adding the heatmap layer
fig.add_layer(heat_layer)



# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Creating heatmap for % humidity
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Creating heatmap for % cloudiness
locations = city_data_df[['Lat','Lng']]
cloudiness = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=cloudiness,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Creating heatmap for Wind Speed
locations = city_data_df[['Lat','Lng']]
wind_speed = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=wind_speed,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
# Asking customer the temperature range
min_temps = float(input('What is the minimum temperature you would like for your trip?'))
max_temps = float(input('What is the maximum temperature you would like for your trip?'))

In [47]:
pref_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temps) & (city_data_df['Max Temp'] >= min_temps)]
pref_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Atuona,PF,2021-12-04 01:17:14,-9.8000,-139.0333,78.87,71,1,16.24
9,9,Brownsville,US,2021-12-04 01:17:17,25.9018,-97.4975,75.09,89,1,4.00
10,10,Mahebourg,MU,2021-12-04 01:17:18,-20.4081,57.7000,77.29,83,40,3.44
14,14,Butaritari,KI,2021-12-04 01:17:21,3.0707,172.7902,82.13,75,93,11.48
15,15,Batticaloa,LK,2021-12-04 01:17:21,7.7102,81.6924,76.82,87,100,4.12


In [55]:
# Checking for null values
pref_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [56]:
#Creating hotel map DataFrame
hotel_df = pref_cities_df[['City','Country', 'Max Temp','Lat','Lng']]
hotel_df['Hotel Name'] = ''
hotel_df.head()

<ipython-input-56-bd3f63e81968>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,78.87,-9.8000,-139.0333,
9,Brownsville,US,75.09,25.9018,-97.4975,
10,Mahebourg,MU,77.29,-20.4081,57.7000,
14,Butaritari,KI,82.13,3.0707,172.7902,
15,Batticaloa,LK,76.82,7.7102,81.6924,


In [62]:
# Setting search parameters for hotels
params = {
    'radius':5000,
    'type':'lodging',
    'key':g_key
}

# Iterating through the DataFrame
for index, row in hotel_df.iterrows():
    # Getting the Lat and Long
    lat = row['Lat']
    lng = row['Lng']

    #Adding the lat and lng to the location key for the params dict
    params['location'] = f'{lat},{lng}'

    # Using the search term: lodging and our lat and long
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #Making request
    hotels = requests.get(base_url,params=params).json()

    #Grabbing the first hotel from the results and storing the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [63]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Atuona,PF,78.87,-9.8000,-139.0333,Villa Enata
9,Brownsville,US,75.09,25.9018,-97.4975,University Inn & Suites
10,Mahebourg,MU,77.29,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
14,Butaritari,KI,82.13,3.0707,172.7902,Isles Sunset Lodge
15,Batticaloa,LK,76.82,7.7102,81.6924,Grand Star Guest House
20,Rikitea,PF,76.93,-23.1203,-134.9692,People ThankYou
21,Avarua,CK,75.25,-21.2078,-159.7750,Paradise Inn
30,Balsas,BR,76.35,-7.5325,-46.0356,Novo Hotel
37,Waingapu,ID,83.46,-9.6567,120.2641,Pondok Wisata Elim
41,Kavaratti,IN,81.09,10.5669,72.6420,Hotels in Lakshadweep Islands


In [67]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Storing the dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [69]:
# Adding a heatmap of temperature for the vacation spots.
locations = hotel_df[['Lat','Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))